In [1]:
import hashlib
import socket
import threading
import string    
import random


In [9]:
#Private
privateString = str("lfAtVRDVhEsIilyjCTKo")
print("privateString: " + str(privateString))
print("Encoded: ", privateString.encode())

result = hashlib.sha1(privateString.encode())
print("SHA1 Hashed: ", result)
print("Hexadecimal: ", result.hexdigest())
 
# Random
randomString = str(''.join(random.choices(string.ascii_letters, k=20)))
print("randomString: ", randomString)
print("Encoded: ", randomString.encode())

result2 = hashlib.sha1(randomString.encode())
print("SHA1 Hashed: ", result2)
print("Hexadecimal: ", result2.hexdigest())

# Concatenated
sha1Result = hashlib.sha1((randomString+privateString).encode())
print("SHA1 Hashed: ", sha1Result)
print("Hexadecimal: ", sha1Result.hexdigest())

privateString: lfAtVRDVhEsIilyjCTKo
Encoded:  b'lfAtVRDVhEsIilyjCTKo'
SHA1 Hashed:  <sha1 _hashlib.HASH object @ 0x0000015EE9B4D170>
Hexadecimal:  5ea54def00326916d15208ce14460b5d0e49632e
randomString:  gphehbCwWlRDGzFqvSKt
Encoded:  b'gphehbCwWlRDGzFqvSKt'
SHA1 Hashed:  <sha1 _hashlib.HASH object @ 0x0000015EE9B4D110>
Hexadecimal:  41b519c480088a1c21012cf9e08a66e94749bae6
SHA1 Hashed:  <sha1 _hashlib.HASH object @ 0x0000015EE9B4D0F0>
Hexadecimal:  db4c1b28a2e09145c06dd827f6aca8c87b7f337d


In [10]:
class ServerSocket:
    def __init__(self, HOST, PORT):
        self.HOST = HOST
        self.PORT = PORT
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket.bind((HOST, PORT))

    def start(self):
        self.socket.listen()
        print(f'[STARTING] {self.HOST}:{self.PORT}')

        while True:
            conn, addr = self.socket.accept()
            print(f'[NEW CONNECTION] {addr}')

            thread = threading.Thread(target=self.handle_connection, args=(conn, addr))
            thread.start()
            print(f'[ACTIVE THREADS] {threading.activeCount()-1}')


    def handle_connection(self, conn, addr):
        connected = True
        while connected:
            data = conn.recv(1024)
            if data:
                msg = data.decode()
                print(f'[NEW MESSAGE] {addr}: {data}')

                if msg == "DISCONNECT":
                    connected = False
                    print(f'[DISCONNECTED] {addr}')

        conn.close()  
        

        

In [11]:
if __name__=="__main__":
    s = ServerSocket('127.0.0.1', 3001)
    s.start()


[STARTING] 127.0.0.1:3001
[NEW CONNECTION] ('127.0.0.1', 60084)
[ACTIVE THREADS] 6
[NEW MESSAGE] ('127.0.0.1', 60084): b'Hello !!!'
[NEW MESSAGE] ('127.0.0.1', 60084): b'DISCONNECT'
[DISCONNECTED] ('127.0.0.1', 60084)
[NEW CONNECTION] ('127.0.0.1', 60085)[ACTIVE THREADS] 6

[NEW MESSAGE] ('127.0.0.1', 60085): b'Hello !!!'
[NEW MESSAGE] ('127.0.0.1', 60085): b'DISCONNECT'
[DISCONNECTED] ('127.0.0.1', 60085)
